In [ ]:
# # Spraw, aby wykresy pojawiały się bezpośrednio w notatniku
# %matplotlib inline

# Import biblioteki do obliczeń numerycznych
import numpy as np

# Wyłączenie komunikatów o błędach numerycznych (dzielenie przez 0 itp.)
np.seterr(all='ignore')

# Import biblioteki do analizy danych
import pandas as pd

# Pobieranie danych z internetu
from pandas_datareader.data import DataReader

# Import biblioteki do numerycznej optymalizacji (max, min, miejsca zerowe)
import scipy.optimize as opt

# Import biblioteki do tworzenia wykresów
import matplotlib.pyplot as plt

# Import biblioteki ekonometrycznej
import statsmodels.api as sm

### Wpływ niepewności na inwestycje

Wiemy z części wykładowej, że w przypadku niepewnej przyszłości poprawna stopa
dyskonta wynika ze stochastycznego czynnika dyskontowego gospodarstw
domowych (akcjonariuszy):
\begin{align*}
p_{t}^{s} & =d_{t}^{s}+\mathrm{E}_{t}\left[\beta\frac{c_{t}}{c_{t+1}}p_{t+1}^{s}\right]\\
V_{t} & =D_{t}+\mathrm{E}_{t}\left[\beta\frac{c_{t}}{c_{t+1}}V_{t+1}\right]
\end{align*}

Rozwiążemy teraz prosty model dwuokresowy, który zilustruje wpływ
niepewności na inwestycje. Zwrot z inwestycji będzie zależał od poziomu
zmiennej losowej $z_{t+1}$ o wartości oczekiwanej 1. Dla uproszczenia
firma przestanie istnieć na koniec okresu $t+1$ a jej kapitał w pełni
się zdeprecjonuje ($\delta=1$). Choć można rozbić problem decyzyjny
na decyzje gospodarstw domowych i firm, najprościej będzie założyć,
że to gospodarstwa domowe (akcjonariusze) bezpośrednio „wybierają”
poziom inwestycji firmy, rozwiązując problem (tu $b$ są obligacjami
np. rządowymi dostępnymi dla gospodarstw domowych jako alternatywna alokacja portfela):
\begin{align*}
\max\quad & U=\frac{c_{t}^{1-\sigma}}{1-\sigma}+\beta\mathrm{E}_{t}\left[\frac{c_{t+1}^{1-\sigma}}{1-\sigma}\right]\\
\text{pod warunkami}\quad & c_{t}+b_{t}+k_{t+1}=y_{t}\\
 & c_{t+1}=\left(1+r\right)b_{t}+z_{t+1}k_{t+1}^{\alpha}
\end{align*}

Funkcja Lagrange'a:
\begin{align*}
\mathcal{L}=\frac{c_{t}^{1-\sigma}}{1-\sigma}+\beta\mathrm{E}_{t}\left[\frac{c_{t+1}^{1-\sigma}}{1-\sigma}\right]+\lambda_{t}\left[y_{t}-c_{t}-b_{t}-k_{t+1}\right]+\mathrm{E}_{t}\left[\beta\lambda_{t+1}\left[\left(1+r\right)b_{t}+z_{t+1}k_{t+1}^{\alpha}-c_{t+1}\right]\right]
\end{align*}

Warunki pierwszego rzędu:
\begin{align*}
c_{t} & : &  & c_{t}^{-\sigma}-\lambda_{t}=0 &  & \to & \lambda_{t} & =c_{t}^{-\sigma}\\
c_{t+1} & : &  & \beta\mathrm{E}_{t}\left[c_{t+1}^{-\sigma}\right]-\mathrm{E}_{t}\left[\beta\lambda_{t+1}\right] &  & \to & \lambda_{t+1} & =c_{t+1}^{-\sigma}\\
b_{t} & : &  & -\lambda_{t}+\mathrm{E}_{t}\left[\beta\lambda_{t+1}\left(1+r\right)\right]=0 &  & \to & \lambda_{t} & =\beta\left(1+r\right)\mathrm{E}_{t}\left[\lambda_{t+1}\right]\\
k_{t+1} & : &  & -\lambda_{t}+\mathrm{E}_{t}\left[\beta\lambda_{t+1}\cdot\alpha z_{t+1}k_{t+1}^{\alpha-1}\right]=0 &  & \to & \lambda_{t} & =\beta\mathrm{E}_{t}\left[\lambda_{t+1}\cdot\alpha z_{t+1}k_{t+1}^{\alpha-1}\right]
\end{align*}

Obligacyjne równanie Eulera:
\begin{align*}
c_{t}^{-\sigma}=\beta\left(1+r\right)\mathrm{E}_{t}\left[c_{t+1}^{-\sigma}\right]
\end{align*}

Akcyjne równanie Eulera:
\begin{align*}
c_{t}^{-\sigma}=\beta\mathrm{E}_{t}\left[c_{t+1}^{-\sigma}\cdot\alpha z_{t+1}k_{t+1}^{\alpha-1}\right]
\end{align*}

Optymalne inwestycje w $k_{t+1}$ muszą spełniać warunek:
\begin{align*}
\left(1+r\right)\mathrm{E}_{t}\left[c_{t+1}^{-\sigma}\right]=\mathrm{E}_{t}\left[c_{t+1}^{-\sigma}\cdot\alpha z_{t+1}k_{t+1}^{\alpha-1}\right]
\end{align*}

W warunkach pewności mielibyśmy $z_{t+1}=1$ oraz:
\begin{align*}
\left(1+r\right) & =\alpha k_{t+1}^{\alpha-1}\\
k_{t+1}^{*} & =\left(\frac{\alpha}{1+r}\right)^{1/\left(1-\alpha\right)}
\end{align*}

Wtedy $c_{t}$ i $c_{t+1}$ spełniałyby równanie Eulera w warunkach
pewności:
\begin{align*}
c_{t+1}=\left[\beta\left(1+r\right)\right]^{1/\sigma}c_{t}
\end{align*}

A obligacje / dług $b_{t}$ dostosowywałyby się tak, aby spełnić ograniczenia
budżetowe:
\begin{align*}
b_{t}=y_{t}-c_{t}^{*}-k_{t+1}^{*}
\end{align*}

W warunkach niepewności problem rozwiążemy numerycznie. Aby jednak
rozwiązanie było intuicyjne, założymy, że: 
\begin{align*}
z_{t+1}=\begin{cases}
1+z & \text{z prawdopodobieństwem }1/2\\
1-z & \text{z prawdopodobieństwem }1/2
\end{cases}
\end{align*}

czyli nasza niepewność będzie zależeć od „wariancji” w postaci $z\in\left(0,1\right)$.


In [ ]:
# Parametry
α	= 0.5		# Elastyczność funkcji produkcji względem kapitału
β	= 0.95		# Czynnik dyskontujący
σ	= 2			# Awersja do ryzyka
y	= 1.0		# Początkowy zasób
r	= 0.02		# Oprocentowanie obligacji

q	= 0.5		# Prawdopodobieństwo niskiego / złego stanu świata

def solve_optimal_kb(z):
	z_hi, z_lo	= 1+z, 1-z	# Przyszłe produktywności

	def Eulers(x):
		k, b	= x
		c1		= y - k - b
		c2_lo	= z_lo * k**α + (1+r) * b
		c2_hi	= z_hi * k**α + (1+r) * b

		# Warunki pierwszego rzędu
		Euler_b	= c1**(-σ) - β * (1+r) * (q * c2_lo**(-σ) + (1-q) * c2_hi**(-σ))
		Euler_k = c1**(-σ) - β * (q * c2_lo**(-σ) * α * z_lo * k**(α-1) + (1-q) * c2_hi**(-σ) * α * z_hi * k**(α-1))
		return [Euler_b, Euler_k]

	sol = opt.root(Eulers, [y/3, y/3])

	if sol.success != 1:
		sol = opt.root(Eulers, [y/2, y/10])

	return sol.x

In [ ]:
# Dla rozmaitych wartości "wariancji" z
z_vals	= np.linspace(0, 1, 100+1)
k_vals	= []
b_vals	= []
c_vals	= []

for z in z_vals:
	k_star, b_star = solve_optimal_kb(z)
	k_vals.append(k_star)
	b_vals.append(b_star)
	c_vals.append(y - b_star - k_star)

# Wykres
plt.stackplot(z_vals, k_vals, b_vals, c_vals, labels=['$k_{t+1}$', '$b_{t}$', '$c_{t}$'])
plt.ylim(0, 1)
plt.xlim(0, z_vals[-1])
plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
plt.title('Optymalna alokacja w warunkach niepewności')
plt.xlabel('Parametr "wariancji" z')
plt.legend(ncol=3, loc='upper center', framealpha=1)
plt.show()

print(f'Przy braku niepewności: k = {k_vals[0]:.2f}, b = {b_vals[0]:.2f}, c_t = {c_vals[0]:.2f}')

In [ ]:
# Wykres równania Eulera
z				= 0.4		# Wartość "realistyczna" to 0.2
z_hi, z_lo		= 1+z, 1-z
k_star, b_star	= solve_optimal_kb(z)

# Poziomy konsumpcji
c1		= y - k_star - b_star
c2_lo	= z_lo * k_star**α + (1+r) * b_star
c2_hi	= z_hi * k_star**α + (1+r) * b_star

# Krańcowe użyteczności
u1_p	= c1**(-σ)
u2_hi_p	= β * (1+r) * c2_hi**(-σ)
u2_lo_p	= β * (1+r) * c2_lo**(-σ)
E_u2_p	= β * (1+r) * (q * c2_lo**(-σ) + (1-q) * c2_hi**(-σ))

# Wykresy
fig, axes = plt.subplots(1, 2, figsize=(12,5))

# Po lewej: krańcowa użyteczność w okresie t
c_grid	= np.linspace(0.4, 1.0, 200)
up_grid	= c_grid**(-σ)

axes[0].plot(c_grid, up_grid, lw=2, label=r'$c_{t}^{-\sigma}$')
axes[0].axhline(E_u2_p, color='C2', linestyle='-', label=r'$\beta(1+r)\mathrm{E}[u^{\prime}(c_{t+1})]$')
axes[0].plot(c1, u1_p, 'C0o')
axes[0].set_xlabel('$c_t$')
axes[0].set_ylabel('(Oczekiwana) krańcowa użyteczność konsumpcji')
axes[0].set_title(r'$u^{\prime}(c_{t})$')
axes[0].legend()
# axes[0].grid(True)

# Po prawej: krańcowa użyteczność w okresie t
axes[1].plot(c_grid, β * (1+r) * up_grid, color='C1', lw=2, label=r'$\beta(1+r)c_{t+1}^{-\sigma}$')
axes[1].plot(c2_lo, u2_lo_p, 'C3o', label='niskie $z$')
axes[1].plot(c2_hi,  u2_hi_p, 'C2o', label='wysokie $z$')
axes[1].axhline(E_u2_p, color='C2', linestyle='-', label='')
axes[1].plot([c2_lo, c2_hi], [u2_lo_p, u2_hi_p], 'C4-')

# Markery wartości oczekiwanych
axes[1].plot(q * c2_lo + (1-q) * c2_hi, E_u2_p, 'C4o', label=r'$\mathrm{E}[u^{\prime}(c_{t+1})]$')
axes[1].plot(q * c2_lo + (1-q) * c2_hi, 
			 β * (1+r) * (q * c2_lo + (1-q) * c2_hi)**(-σ), 'C9o', label=r'$u^{\prime}(\mathrm{E}[c_{t+1}])$')

axes[1].set_xlabel(r'$c_{t+1}$')
axes[1].set_title(r'$\beta(1+r)u^{\prime}(c_{t+1})$')
axes[1].legend()

# Dopasuj limity osi y
ymin = min(axes[0].get_ylim()[0], axes[1].get_ylim()[0])
ymax = max(axes[0].get_ylim()[1], axes[1].get_ylim()[1])
axes[0].set_ylim(ymin, ymax)
axes[1].set_ylim(ymin, ymax)

plt.suptitle(f'Równanie Eulera dla z = {z:.1f} i σ = {σ:.1f}')
plt.tight_layout()
plt.show()

# Wydruk rozwiązania
print(f"Optymalne k = {k_star:.2f}, b = {b_star:.2f}, c_t = {c1:.2f}, c_t+1_l = {c2_lo:.2f}, c_t+1_h = {c2_hi:.2f}\n")
print(f"u'(c_t) = {u1_p:.2f}, E[β(1+r)u'(c_t+1)] = {E_u2_p:.2f}\n")
print(f'Oczekiwana stopa zwrotu z kapitału = {100*(α * k_star**(α-1) - 1):.2f} %')
print(f'Premia za ryzyko                   = {100*np.abs(α * k_star**(α-1) - 1 - r):.2f} %')

### Szacowanie modelu wektorowej autoregresji (VAR)

In [ ]:
start	= '1947-01'
end		= '2030-01'

#				Inwestycje	Wycena rynkowa	Wartość księgowa netto
try:
	fred = DataReader(['GPDIC1', 'NCBEILQ027S', 'TNWMVBSNNCB'], 'fred', start=start, end=end)
	fred.to_csv('Dane/Inwestycje.csv')
except:
	fred = pd.read_csv('Dane/Inwestycje.csv')

fred['Q'] = fred['NCBEILQ027S'] / fred['TNWMVBSNNCB'] / 1000

In [ ]:
# Rzut oka na dane

fig, ax = plt.subplots()

fred['GPDIC1'].plot(ax=ax, logy=True, label='Realne zagregowane inwestycje (oś lewa)', lw=2)
fred['Q'].plot(ax=ax, secondary_y=True, logy=True, lw=2, color='C3', label='Przeciętne Q (oś prawa)')

ax.set_ylim(150, 10000)
ax.set_yticks([200, 500, 1000, 2000, 5000])
ax.set_yticklabels([200, 500, 1000, 2000, 5000])

ax.right_ax.set_ylim(5e-2, 2.5)
ax.right_ax.set_yticks([0.2, 0.5, 1, 1.5, 2])
ax.right_ax.set_yticklabels([0.2, 0.5, 1, 1.5, 2])

lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax.right_ax.get_legend_handles_labels()
ax.legend(lines + lines2, labels + labels2, loc='lower right', frameon=False)

ax.set_xlabel('')
plt.show()

In [ ]:
# Transformacja logarytmiczna
dta = pd.DataFrame()
dta['Inv_L'] = 100*np.log(fred['GPDIC1'])
dta['Q_L'] = 100*np.log(fred['Q'])

# Jednostronny filtr HP
from hpfilter import *

dta['Q_c'], dta['Q_t'] = hprescott(dta['Q_L'].dropna(), 1, 1600)
dta['I_c'], dta['I_t'] = hprescott(dta['Inv_L'].dropna(), 1, 1600)

In [ ]:
# Dane i trend HP: Q

fig, ax = plt.subplots()

dta['Q'] = np.exp(dta['Q_L']/100)
dta['Q_tt'] = np.exp(dta['Q_t']/100)

dta[['Q', 'Q_tt']].plot(ax=ax, logy=True, style=['C0', 'C3'], lw=2)
(dta['Inv_L']**0).plot(ax=ax, lw=1, style='k')

plt.legend(['Przeciętne Q', 'Trend HP (jednostronny)'], loc='lower center', ncol=2, frameon=False)

ax.set_ylim(0.2, 2)
ax.set_yticks([0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2])
ax.set_yticklabels([0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1, 1.2, 1.4, 1.6, '', 2])

ax.set_xlabel('')
plt.show()

In [ ]:
# Dane i trend HP: inwestycje

fig, ax = plt.subplots()

dta['Inv'] = np.exp(dta['Inv_L']/100)
dta['I_tt'] = np.exp(dta['I_t']/100)

dta[['Inv', 'I_tt']].plot(ax=ax, logy=True, style=['C0', 'C3'], lw=2)
plt.legend(['Realne zagregowane inwestycje', 'Trend HP (jednostronny)'], loc='upper left', frameon=False)

ax.set_ylim(150, 5000)
ax.set_yticks([200, 500, 1000, 2000, 5000])
ax.set_yticklabels([200, 500, 1000, 2000, 5000])

ax.set_xlabel('')
plt.show()

In [ ]:
# Odchylenia od trendu
dta[['Q_c', 'I_c']].plot(style=['C0', 'C3'], lw=2)
plt.show()

In [ ]:
# Dane do regresji VAR
df = pd.DataFrame()
df[' Q '] = dta['Q_c']
df[' Investment '] = dta['I_c']
df = df.dropna()

# Szacowanie modelu na danych sprzed pandemii
model = sm.tsa.VAR(df['1990':'2019-12-31'], freq='QS')

# Kryteria selekcji opóźnień
print(model.select_order(12))

# AIC oraz FPE osiągają max przy 2, ale 4 to "ładny" rok, więc to wybieram
results = model.fit(4)

In [ ]:
results.summary()

In [ ]:
# Test na przyczynowość w sensie Grangera
print(results.test_causality(' Q ', ' Investment ').summary())
print('')
print(results.test_causality(' Investment ', ' Q ').summary())


In [ ]:
# Funkcje reakcji na impuls (Impulse Response Functions)
results.irf(12).plot(orth=True, signif=0.05, figsize=(8,8), subplot_params={'fontsize' : 14})
plt.show()

In [ ]:
# Dekompozycja wariancji błędów prognozy
results.fevd(20).plot(figsize=(8,8))
plt.show()

In [ ]:
# Prognozowanie nie jest zbyt przyjaźnie zakodowane...
results.plot_forecast(4)
plt.show()

In [ ]:
# "Ręczne" generowanie prognoz
lag_order = results.k_ar

fcast = []

for t in reversed(range(1, len(df.dropna().index)-lag_order)):
	fcast.append(results.forecast(df.values[-lag_order-t:-t], 1)[0][1])
	
fcast.append(results.forecast(df.values[-lag_order:], 1)[0][1])

# Modyfikacja dta
dta['I_c_f'] = np.nan

dta['I_c_f'][len(dta.index)-len(fcast):] = fcast

dta['Inv_f'] = np.exp((dta['I_t']+dta['I_c_f'])/100)

In [ ]:
# Prognoza odchyleń od trendów na 1 kwartał do przodu
dta[['I_c', 'I_c_f']].plot(style=['C0', 'C3'], lw=2)
plt.show()

In [ ]:
fig, ax = plt.subplots()

dta[['Inv', 'Inv_f']].plot(ax=ax, logy=True, style=['C0', 'C3'], lw=2)
plt.legend(['Realne zagregowane inwestycje', 'Prognoza na 1 kwartał do przodu'], loc='upper left', frameon=False)

ax.set_ylim(150, 5000)
ax.set_yticks([200, 500, 1000, 2000, 5000])
ax.set_yticklabels([200, 500, 1000, 2000, 5000])

ax.set_xlabel('')
plt.show()

In [ ]:
fig, ax = plt.subplots()

dta[['Inv', 'Inv_f']]['1999':].plot(ax=ax, style=['C0', 'C3'], lw=2)
plt.legend(['Realne zagregowane inwestycje', 'Prognoza na 1 kwartał do przodu'], frameon=False)

plt.ylim(1750, 4750)
plt.yticks(np.arange(1750, 5000, 250))

ax.set_xlabel('')
plt.show()

In [ ]:
RMSE = np.sqrt( np.average( (100*(dta['Inv_f']['1990':] / dta['Inv']['1990':] - 1)).dropna()**2 ) )
RMSE_in = np.sqrt( np.average( (100*(dta['Inv_f']['1990':'2019-12-31'] / dta['Inv']['1990':'2019-12-31'] - 1)).dropna()**2 ) )
RMSE_out = np.sqrt( np.average( (100*(dta['Inv_f']['2020':] / dta['Inv']['2020':] - 1)).dropna()**2 ) )
print(f'Błąd średniokwadratowy prognozy (1990-2025) = {RMSE:.2f}')
print(f'Błąd średniokwadratowy prognozy (1990-2019) = {RMSE_in:.2f}')
print(f'Błąd średniokwadratowy prognozy (2020-2025) = {RMSE_out:.2f}')

#### Model autoregresyjny (AR) dla inwestycji bez informacji o Q

In [ ]:
# Autoregresja jednej zmiennej
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

df_AR = df[' Investment ']['1990':'2019-12-31'].dropna()

# Szacowanie modelu AR na danych sprzed pandemii
model_alt = AutoReg(df_AR, 12).fit()
print(model_alt.summary())

In [ ]:
# Są ładne wykresy wyników regresji AR
fig = plt.figure(figsize=(10, 8))
model_alt.plot_diagnostics(fig=fig)
plt.show()

In [ ]:
# Prognozy z modelu AR (tylko in sample, out of sample trzeba liczyć "ręcznie")
dta['I_c_f_AR'] = model_alt.predict()

dta['Inv_f_AR'] = np.exp((dta['I_t']+dta['I_c_f_AR'])/100)

In [ ]:
fig, ax = plt.subplots()

dta[['Inv', 'Inv_f_AR']].plot(ax=ax, logy=True, style=['C0', 'C3'], lw=2)
plt.legend(['Realne zagregowane inwestycje', 'Prognoza na 1 kwartał do przodu (AR)'], loc='upper left', frameon=False)

ax.set_ylim(150, 5000)
ax.set_yticks([200, 500, 1000, 2000, 5000])
ax.set_yticklabels([200, 500, 1000, 2000, 5000])

ax.set_xlabel('')
plt.show()

In [ ]:
fig, ax = plt.subplots()

dta[['Inv', 'Inv_f_AR']]['1999':].plot(ax=ax, style=['C0', 'C3'], lw=2)
plt.legend(['Realne zagregowane inwestycje', 'Prognoza na 1 kwartał do przodu (AR)'], frameon=False)

plt.ylim(1750, 4750)
plt.yticks(np.arange(1750, 5000, 250))

ax.set_xlabel('')
plt.show()

In [ ]:
RMSE_AR = np.sqrt( np.average( (100*(dta['Inv_f_AR']['1990':] / dta['Inv']['1990':] - 1)).dropna()**2 ) )

print(f'Błąd średniokwadratowy prognozy AR  (1990-2019) = {RMSE_AR:.2f}%')
print(f'Błąd średniokwadratowy prognozy VAR (1990-2019) = {RMSE_in:.2f}%')

In [ ]:
import seaborn as sns

sns.kdeplot(100*(dta['Inv_f']['1990':] / dta['Inv']['1990':] - 1), fill=True, label='Model VAR')
sns.kdeplot(100*(dta['Inv_f_AR']['1990':] / dta['Inv']['1990':] - 1), fill=True, label='Model AR')
plt.title('Rozkłady błędów prognozy (w %)')
plt.ylabel('Gęstość')
plt.legend(frameon=False)
plt.show()